# AMLS_assignment20_21/ SN:14066539

==============================================================================================

#### If you come across memory issue, my advise would be to change Target_size in the relevant files (MA1, MA2, MB1, MB2)

## Importin required libraries

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as v1
import numpy as np
import os
from keras.preprocessing import image
import scipy.io
import pandas as pd
from matplotlib import pyplot
import matplotlib as mpl
import matplotlib.pyplot as plt
from A1 import MA1
from B1 import MB1
from A2 import MA2
from B2 import MB2
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

==============================================================================================

## Mapping of directories

In [ ]:
global data_dir, image_dir, label_dir, asset_dir

data_dir = ('Datasets')
image_dir_celeba = os.path.join(data_dir, 'celeba', 'img')
image_dir_cartoonset = os.path.join(data_dir,'cartoon_set', 'img')
asset_dir_celeba = os.listdir(os.path.join(os.getcwd(), 'Datasets/celeba/img'))
asset_dir_cartoonset = os.listdir(os.path.join(os.getcwd(), 'Datasets/cartoon_set/img'))
label_dir_celeba = os.path.join(data_dir,'celeba', 'labels.csv')
label_dir_cartoonset = os.path.join(data_dir, 'cartoon_set', 'labels.csv')

In [ ]:
# Setting directories global for use across multiple models
global data_dir, image_dir_celeba, image_dir_cartoonset, asset_dir_celeba, labels_dir_celeba, labels_dir_cartoonset

==============================================================================================

# MA1 k-Means & MLP model for gender recognition : celeba dataset

1000 images from celeba dataset was initially conversted to train MultiLayer Perceptron model. 

### 1. Running k_Means function in order to get centroids coordinates with relevant arguments

In [ ]:
# specify criteria
max_iters = 10         # Number of iterations, ideal to keep it <10 
samples = 100          # It is recommended to keep this number small (in the range of 10 - 30), as this calculation is computationally expensive process.
K = 20                 # Number of centroids. Not all the images high dimension data available to slice, hence as per my observation, a number between 10 and 50 is ideal for this dataset.
target_size = (79, 59)  # multiply this three times to make it actual size.

image_dir_celeba = os.path.join(data_dir, 'celeba', 'img')
asset_dir_celeba = os.listdir(os.path.join(os.getcwd(), 'Datasets/celeba/img'))

# Get the centroids of the sample images --------------------
centroid_array, idx_array = MA1.get_centroids_celebaMA1(asset_dir_celeba, image_dir_celeba, samples, K, max_iters, target_size)

###### Please ignore the alert of empty slice. It seems to me that the dataset was already optimized, hence the error indicates lack of centroids in different dimensions.

#### 1.1 Shapping and loading data for MLP

In [ ]:
# Getting shapped data for Model A1:

number_labels = len(set(pd.read_csv(label_dir_celeba, delimiter= '\t' )['gender']))

X_train, X_test, Y_train, Y_test, features = MA1.get_data_MA1(number_labels)

### 2. Setting up MLP with effective parameters

In [ ]:
# Number of hidden layers
n_hidden_1 = 2048 # can change this number to optimize results
n_hidden_2 = 2048 # can change this number to optimize results

# Setting up learning rates and number of epochs for start
learning_rate = 1e-5
training_epochs = 500
display_accuracy_step = 10

weights, X, Y, biases, input_array = MA1.initialize_parameters(X_train, Y_train, n_hidden_1, n_hidden_2)

logits = MA1.multilayer_perceptron(weights, biases, input_array)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))

optimizer = v1.train.AdamOptimizer(learning_rate).minimize(loss_op)

#### 2.1  Running MLP model

In [ ]:
costs = []

init_op = v1.global_variables_initializer() # initializing global parameter

with v1.Session() as sess:

    # run graph weights/biases initialization op
    sess.run(init_op)

    for epoch in range(training_epochs):

        # running cost function (backprop)
        _, cost = sess.run([optimizer, loss_op], feed_dict={X: X_train, Y: Y_train})
        costs.append(cost)

        # Plotting results
        
        if epoch % display_accuracy_step == 0:
            predict = tf.nn.softmax(logits)  # Softmax
            correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            # on using argmax to predict accuracy on axis = 1
            # calculate training accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
            test_accuracy = accuracy.eval({X: X_test, Y: Y_test})

    # Plotting results
    
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    print (" .. Model Finished ..")

In [ ]:
costs[0]

In [ ]:
costs[-1]

### Demonstration of k-Means compression on an image

In [ ]:
# ======= Experiment with these parameters ================
K = 24
max_iters = 10

# Change the file name and path to experiment with different images
img_sample = os.path.join(image_dir_celeba, '1.jpg')
A = image.img_to_array(image.load_img(img_sample, target_size = None , interpolation='bicubic'))

# Divide by 255 so that all values are in the range 0 - 1
A = A / 255
X = A.reshape(-1, 3)

initial_centroids = MA1.kMeansInitCentroids(X, K)
centroids, idx = MA1.runkMeans(X, initial_centroids,
                                 MA1.findClosestCentroids,
                                 MA1.computeCentroids,
                                 max_iters)

X_recovered = centroids[idx, :].reshape(A.shape)


# Display the original image, rescale back by 255
fig, ax = pyplot.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(A)
ax[0].set_title('Original')
ax[0].grid(False)

# Display compressed image, rescale back by 255
ax[1].imshow(X_recovered)
ax[1].set_title('Compressed, with %d colors' % K)
ax[1].grid(False)

==============================================================================================

# MA2 Principal Component Analysis (PCA) & MLP for Smiling recognition : celeba dataset

This model focuses on analysing Celeba datasets using Principal Component. 
First stage of implementation of this algorithm is to find the correct set of Principal Component (eigenfaces) that describes the largest variations of the image. This can be changed by increasing K in this model. But due to the computational restrictions, I have used 128.

This algorithm is implemented in two steps.
(i) first we normalize the dataset by subtracting the mean of each features from the matrix.
(ii)In this stage we can use normalized values to reduce the dimesnsion fo the dataset. Which allows learning algorithm with a smaller input size.

In this Model I have used K = 128. The K here was derived using covariance matrix greater than 1. 

### 1. PCA compression function

In [ ]:
def PCA_compression(K, number_labels): # mention of K depending on the image resolution and criteria
    
    X, Y = MA2.data_to_vector_MA2 (asset_dir_celeba, number_labels)
    
    X_norm, mu, sigma = MA2.featureNormalize(X)
    U, S = MA2.pca(X_norm)
    X = MA2.projectData(X_norm, U, K)
    
    X_train, X_test, Y_train, Y_test, X_val, Y_val = MA2.split_data(X, Y)
    print('Shape of compressed X_train:', X_train.shape)
    print('Shape of compressed X_test:', X_test.shape)
    print('Shape of Y_train:', Y_train.shape)
    print('Shape of Y_test:', Y_test.shape)
    print('Shape of X_val:', X_val.shape)
    print('Shape of Y_val:', Y_val.shape)
    
    return X_train, X_test, Y_train, Y_test, X_val, Y_val, U, S

#### 1.1 Compressing and loading compressed image data

In [ ]:
K = 128    # I have chosen value of 128 as it gives high degree of accuracy perhaps way too much for this purpose :)

number_labels = len(set(pd.read_csv(label_dir_celeba, delimiter= '\t' )['smiling']))

# Current Image size is set to (79, 59), this will need to change as required in file MA2.py (function: data_to_vector_MA2)

X_train, X_test, Y_train, Y_test, X_val, Y_val, U, S = PCA_compression(K, number_labels)

### 2. Setting up MLP with basic arguments

In [ ]:
# Number of hidden layers
n_hidden_1 = 2048 # can change this number to optimize results
n_hidden_2 = 2048 # can change this number to optimize results

# Setting up learning rates and number of epochs for start
learning_rate = 1e-5
training_epochs = 200
display_accuracy_step = 10

weights, X, Y, biases, input_array = MA2.initialize_parameters(X_train, Y_train, n_hidden_1, n_hidden_2)

logits = MA2.multilayer_perceptron(weights, biases, input_array)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))

optimizer = v1.train.AdamOptimizer(learning_rate).minimize(loss_op)

#### 2.1 Running MLP model

In [ ]:
costs = []

init_op = v1.global_variables_initializer() # initializing global parameter

with v1.Session() as sess:

    # run graph weights/biases initialization op
    sess.run(init_op)

    for epoch in range(training_epochs):

        # running cost function (backprop)
        _, cost = sess.run([optimizer, loss_op], feed_dict={X: X_train, Y: Y_train})
        costs.append(cost)

        # Plotting results
        
        if epoch % display_accuracy_step == 0:
            predict = tf.nn.softmax(logits)  # Softmax
            correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            # on using argmax to predict accuracy on axis = 1
            # calculate training accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
            val_accuracy = accuracy.eval({X: X_val, Y: Y_val})
            test_accuracy = accuracy.eval({X: X_test, Y: Y_test})

    # Plotting results
    
    print("Train Accuracy:", train_accuracy)
    print("Val Accuracy:", val_accuracy)
    print("Test Accuracy:", test_accuracy)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    print (" .. Model Finished ..")

In [ ]:
costs[0]

### ==============================================================

# MB1 PCA & MLP for eye-color recognition : cartoon_set

First stage of implementation of this algorithm is to find the correct set of Principal Component (eigenfaces) that describes the largest variations of the image. This can be changed by increasing K in this model. But due to the computational restrictions, I have used 128.

This algorithm is implemented in two steps across both datasets.
(i) first we normalize the dataset by subtracting the mean of each features from the matrix.
(ii)In this stage we can use normalized values to reduce the dimesnsion fo the dataset. Which allows learning algorithm with a smaller input size.

In this Model I have used K = 128. The K here was derived using covariance matrix greater than 1. 

### 1. PCA compression function

In [ ]:
def PCA_compression(K, number_labels): # mention of K depending on the image resolution and criteria
    
    X, Y = MB1.data_to_vector_MB1 (asset_dir_cartoonset, number_labels)
    
    X_norm, mu, sigma = MB1.featureNormalize(X)
    U, S = MB1.pca(X_norm)
    X = MB1.projectData(X_norm, U, K)
    
    X_train, X_test, Y_train, Y_test, X_val, Y_val = MB1.split_data(X, Y)
    print('Shape of compressed X_train:', X_train.shape)
    print('Shape of compressed X_test:', X_test.shape)
    print('Shape of Y_train:', Y_train.shape)
    print('Shape of Y_test:', Y_test.shape)
    print('Shape of X_val:', X_val.shape)
    print('Shape of Y_val:', Y_val.shape)
    
    return X_train, X_test, Y_train, Y_test, X_val, Y_val, U, S

#### 1.1 Compressing and loading compressed image data

In [ ]:
K = 128    # I have chosen value of 128 as it gives high degree of accuracy perhaps way too much for this purpose :)

number_labels = len(set(pd.read_csv(label_dir_cartoonset, delimiter= '\t' )['eye_color']))

X_train, X_test, Y_train, Y_test, X_val, Y_val, U, S = PCA_compression(K, number_labels)

### 2. Setting up MLP with basic arguments

In [ ]:
# Number of hidden layers
n_hidden_1 = 2048 # can change this number to optimize results
n_hidden_2 = 2048 # can change this number to optimize results

# Setting up learning rates and number of epochs for start
learning_rate = 1e-5
training_epochs = 2000
display_accuracy_step = 10

weights, X, Y, biases, input_array = MB1.initialize_parameters(X_train, Y_train, n_hidden_1, n_hidden_2)

logits = MB1.multilayer_perceptron(weights, biases, input_array)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_op)

#### 2.1 Running MLP model

In [ ]:
costs = []
validation = []

init_op = v1.global_variables_initializer() # initializing global parameter

with v1.Session() as sess:

    # run graph weights/biases initialization op
    sess.run(init_op)

    for epoch in range(training_epochs):

        # running cost function (backprop)
        _, cost = sess.run([optimizer, loss_op], feed_dict={X: X_train, Y: Y_train})
        costs.append(cost)

        # Plotting results
        
        if epoch % display_accuracy_step == 0:
            predict = tf.nn.softmax(logits)  # Softmax
            correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            # on using argmax to predict accuracy on axis = 1
            # calculate training accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
            val_accuracy = accuracy.eval({X: X_val, Y: Y_val})
            test_accuracy = accuracy.eval({X: X_test, Y: Y_test})

    # Plotting results
    
    print("Train Accuracy:", train_accuracy)
    print("Validation Accuracy:", val_accuracy)
    print("Test Accuracy:", test_accuracy)
    plt.plot(np.squeeze(costs))
    plt.plot(np.squeeze(validation))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    print (" .. Model Finished ..")

### ==============================================================

# MB2 K-mean & MLP for face-shape recognition : cartoon_set

### 1. Running k-Means in order to compress data

In [ ]:
# specify criteria
max_iters = 6     # Number of iterations, 
samples = 1000     # It is recommended to keep this number small but could go as large as the whole dataset. This calculation is computationally expensive process.
K = 12            # Number of centroids. Not all the images high dimension data available to slice, hence as per my observation, a number between 10 and 50 is ideal for this dataset.
target_size = (150,150)

# Get the centroids of the sample images --------------------
cartoon_centroid_array, cartoon_idx_array = MB2.get_centroids_cartoonMB2(asset_dir_cartoonset, image_dir_cartoonset, samples, K, max_iters, target_size)

#### 1.1 Shapping and loading data to pass it on to MLP

In [ ]:
# Getting shapped data for Model A1:

number_labels = len(set(pd.read_csv(label_dir_cartoonset, delimiter= '\t')['face_shape'])) # Male or Female

X_train, X_test, Y_train, Y_test, features = MB2.get_data_MB2(number_labels)

### 2. Setting up MLP with effective parameters

In [ ]:
# Number of hidden layers
n_hidden_1 = 2048 # can change this number to optimize results
n_hidden_2 = 2048 # can change this number to optimize results

# Setting up learning rates and number of epochs for start
learning_rate = 1e-5
training_epochs = 1000
display_accuracy_step = 10

weights, X, Y, biases, input_array = MA1.initialize_parameters(X_train, Y_train, n_hidden_1, n_hidden_2)

logits = MA1.multilayer_perceptron(weights, biases, input_array)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))

optimizer = v1.train.AdamOptimizer(learning_rate).minimize(loss_op)

#### 2.1  Running MLP model

In [ ]:
costs = []

init_op = v1.global_variables_initializer() # initializing global parameter

with v1.Session() as sess:

    # run graph weights/biases initialization op
    sess.run(init_op) 

    for epoch in range(training_epochs):

        # running cost function (backprop)
        _, cost = sess.run([optimizer, loss_op], feed_dict={X: X_train, Y: Y_train})
        costs.append(cost)

        # Plotting results
        
        if epoch % display_accuracy_step == 0:
            predict = tf.nn.softmax(logits)  # Softmax
            correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            # on using argmax to predict accuracy on axis = 1
            # calculate training accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
            test_accuracy = accuracy.eval({X: X_test, Y: Y_test})

    # Plotting results
    
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    print (" .. Model Finished ..")

In [ ]:
costs[0]

In [ ]:
costs [-1]